In [152]:
import tensorflow as tf
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [153]:
import io
import re
import os
import tqdm
import shutil
import string
import random
import numpy as np 

import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import spacy
import fr_core_news_sm

from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

from collections import defaultdict
from collections import Counter

import tensorflow as tf
from tensorflow import keras
from keras.models import Model, Sequential
from keras.initializers import Constant
from keras.layers import (Input,
                          LSTM, 
                          Embedding, 
                          BatchNormalization,
                          Dense, 
                          TimeDistributed, 
                          Dropout, 
                          Bidirectional,
                          Flatten, 
                          GlobalMaxPool1D)

from tensorflow.keras import Model
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM, Flatten, Dot
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer

from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    classification_report,
    accuracy_score
)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

In [154]:
import os
import torch
import tensorflow
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import warnings
warnings.filterwarnings("ignore")

torch.__version__, tensorflow.__version__

('1.9.0+cu111', '2.9.1')

In [155]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data-books.json

Maintenant que nous savons que la target doit être encodée en 0 ou 1 (en augmentant la taille des matrices de sortie pour faire correspondre le nombre de valeurs correspondantes), nous allons nous baser sur 2 modèles:</br>
* modèle MultiLabelBinarizer
* modèle custome avec des 0 et 1 (soit 8 vecteurs représentant les 8 sentiments, soit autant de vecteurs que de notes différentes, soit 5 * 8 vecteurs de 0 à 1 (5 représentant une note par sentiment allant de 0 à 4 par exemple)

In [164]:
p = r'C:\Users\david\JEDHA BOOTCAMP\FULLSTACK\10_PROJECT\FINAL_PROJECT\csv_files'
f = os.path.join(p, '', 'data-books.json')

books = pd.read_json(f, lines=True)

In [165]:
# on selectionne seulement les colonnes avec les sentiments
new_books = books[[col for col in books.columns if col.startswith('sen')]]

# on récupère les index des lignes qui contiennent au moins une valeur != de 0 (donc ou il y a des sentiments)
ind = new_books[new_books.any(axis=1)].index

# on verifie sur 3 lignes si nous ne nous sommes pas trompés
display(new_books[new_books.any(axis=1)].head(3))

# books contient seulement les lignes avec les notes des sentiments
books = books.iloc[ind].reset_index(drop=True)
display(books.head(3))

,sen_colère,sen_confiance,sen_honte,sen_joie,sen_peur,sen_réflexion,sen_surprise,sen_tristesse
8,0.052308,0.048318,0.000000,0.184525,0.065355,0.102246,0.240899,0.165582
16,0.049184,0.056415,0.025648,0.408646,0.000000,0.102054,0.122908,0.100963
20,0.076374,0.158335,0.000000,0.196684,0.108542,0.153644,0.196747,0.183018


,book_id,book_url,book_nb_comm,title,name,surname,tags,img_url,book_rating_count,book_rating_value,...,tag_asexualité,tag_aromantisme,tag_licorne,tag_alopécie,tag_héritage,tag_mystère,tag_destin,tag_gangs,tag_littérature_algérienne,tag_identité
0,1578,https://www.babelio.com/livres/Loisel-La-Quete...,42,"La Quête de l'oiseau du temps, tome 1 : La con...",Régis,Loisel,"[['aventure', 17], ['serie', 15], ['saga', 15]...",https://images-na.ssl-images-amazon.com/images...,628,4.06,...,0,0,0,0,0,0,0,0,0,0
1,1700,https://www.babelio.com/livres/Beaumarchais-Le...,85,Le Mariage de Figaro,,Beaumarchais,"[['satire', 15], ['comédie', 19], ['théâtre cl...",https://images-na.ssl-images-amazon.com/images...,1823,3.65,...,0,0,0,0,0,0,0,0,0,0
2,1725,https://www.babelio.com/livres/Buzzati-Le-dese...,205,Le désert des Tartares,Dino,Buzzati,"[['roman', 24], ['fantastique', 17], ['classiq...",https://m.media-amazon.com/images/I/41CPam0DPt...,2435,4.07,...,0,0,0,0,0,0,0,0,0,0


In [166]:
sentiments_cols = [col for col in books.columns if col.startswith('sen')]

for col in sentiments_cols:
    print("Colonne :",col)
    print("Minimum :", books[col].min())
    print("Maximum :", books[col].max(), '\n')

Colonne : sen_colère
Minimum : 0.0
Maximum : 0.529132319 

Colonne : sen_confiance
Minimum : 0.0
Maximum : 0.532775507 

Colonne : sen_honte
Minimum : 0.0
Maximum : 0.2713094573 

Colonne : sen_joie
Minimum : 0.024315695800000002
Maximum : 0.6002598427 

Colonne : sen_peur
Minimum : 0.0
Maximum : 0.2994763939 

Colonne : sen_réflexion
Minimum : 0.0
Maximum : 0.5127873827 

Colonne : sen_surprise
Minimum : 0.0
Maximum : 0.49245117180000003 

Colonne : sen_tristesse
Minimum : 0.0
Maximum : 0.4946098015 



In [167]:
def create_categ(x):
    if x == float(0):
        x = 0
        return int(x)
    if x < 0.10 and x != float(0):
        x = 1
        return int(x)
    if x < 0.20 and x != float(0):   
        x = 2
        return int(x)
    if x < 0.30 and x != float(0): 
        x = 3
        return int(x)
    if x < 0.40 and x != float(0): 
        x = 4
        return int(x)
    if x < 0.50 and x != float(0): 
        x = 5
        return int(x)
    else:
        x = 6
        return int(x)

In [168]:
df = books.copy()
sentiments_cols = [col for col in df.columns if col.startswith('sen')]
columns = ['colere', 'confiance', 'honte', 'joie', 'peur', 'reflexion', 'surprise', 'tristesse']
for i,j in zip(sentiments_cols, columns):
    df[j] = df[i].apply(create_categ)

In [173]:
for i,el in enumerate([el for double in double_liste for el in double]):
    col_books.insert(i+n, el, 0)

In [180]:
col_books[col_books.columns[8:]]

,colere_0,colere_1,colere_2,colere_3,colere_4,colere_5,colere_6,confiance_0,confiance_1,confiance_2,...,surprise_4,surprise_5,surprise_6,tristesse_0,tristesse_1,tristesse_2,tristesse_3,tristesse_4,tristesse_5,tristesse_6
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
904,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
905,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
906,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [228]:
def create_new_columns(df):
    sentiments_cols = [col for col in df.columns if col.startswith('sen')]
    columns = ['colere', 'confiance', 'honte', 'joie', 'peur', 'reflexion', 'surprise', 'tristesse']
    for i,j in zip(sentiments_cols, columns):
        df[j] = df[i].apply(create_categ)
    col_books = df[columns]
    n = len(col_books.columns)
    double_liste = [[col+'_'+str(i) for i in range(7)] for col in col_books[columns].columns]
    for i,el in enumerate([el for double in double_liste for el in double]):
        col_books.insert(i+n, el, 0)
    # print(col_books.columns)
    col_books = pd.concat([df,col_books[col_books.columns[8:]]], axis=1)
    i=0
    while i < col_books.shape[0]-1:
        for col in columns:
            for el in col_books.columns[col_books.columns.str.startswith(col+'_')]:
                if col_books.loc[i,col] == int(el[-1]):
                    col_books.loc[i,el] = 1
        i+=1
        
    return col_books

In [229]:
new_b = create_new_columns(books)
new_b

,book_id,book_url,book_nb_comm,title,name,surname,tags,img_url,book_rating_count,book_rating_value,...,surprise_4,surprise_5,surprise_6,tristesse_0,tristesse_1,tristesse_2,tristesse_3,tristesse_4,tristesse_5,tristesse_6
0,1578,https://www.babelio.com/livres/Loisel-La-Quete...,42,"La Quête de l'oiseau du temps, tome 1 : La con...",Régis,Loisel,"[['aventure', 17], ['serie', 15], ['saga', 15]...",https://images-na.ssl-images-amazon.com/images...,628,4.06,...,0,0,0,0,0,1,0,0,0,0
1,1700,https://www.babelio.com/livres/Beaumarchais-Le...,85,Le Mariage de Figaro,,Beaumarchais,"[['satire', 15], ['comédie', 19], ['théâtre cl...",https://images-na.ssl-images-amazon.com/images...,1823,3.65,...,0,0,0,0,0,1,0,0,0,0
2,1725,https://www.babelio.com/livres/Buzzati-Le-dese...,205,Le désert des Tartares,Dino,Buzzati,"[['roman', 24], ['fantastique', 17], ['classiq...",https://m.media-amazon.com/images/I/41CPam0DPt...,2435,4.07,...,0,0,0,0,0,1,0,0,0,0
3,1829,https://www.babelio.com/livres/Ayroles-De-Cape...,34,"De Cape et de Crocs, tome 2 : Pavillon noir !",Alain,Ayroles,"[['aventure', 19], [""de capes et d'épées"", 18]...",/couv/CVT_CVT_De-Cape-et-de-Crocs-tome-2--Pavi...,429,4.30,...,0,0,0,0,1,0,0,0,0,0
4,2109,https://www.babelio.com/livres/Rowling-Harry-P...,601,"Harry Potter, tome 7 : Harry Potter et les rel...",J. K.,Rowling,"[['aventure', 18], ['saga', 17], ['roman', 21]...",/couv/CVT_10230_671213.jpg,15883,4.54,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,1452627,https://www.babelio.com/livres/-Jamais-plus/14...,399,Jamais plus,Colleen,Hoover,"[['journal intime', 14], ['roman', 19], ['jeun...",https://images-eu.ssl-images-amazon.com/images...,2113,4.38,...,0,0,0,0,0,0,1,0,0,0
904,1453053,https://www.babelio.com/livres/-Grace-and-Fury...,64,"Grace and Fury, tome 1 : Fleurs de fer",Tracy,Banghart,"[['saga', 14], ['aventure', 19], ['roman', 14]...",https://m.media-amazon.com/images/I/516CV-xP6p...,104,4.07,...,0,0,0,0,0,1,0,0,0,0
905,1453136,https://www.babelio.com/livres/-Vers-les-etoil...,64,Vers les étoiles,Mary Robinette,Kowal,"[['uchronie', 21], ['roman', 15], ['dystopie',...",https://m.media-amazon.com/images/I/41JI86XiUj...,168,3.83,...,0,0,0,0,0,1,0,0,0,0
906,1455462,https://www.babelio.com/livres/-Les-gardiens-d...,62,"Les gardiens d'Ülgeriin, tome 1 : Rose éternelle",Ophélie,Duchemin,"[['saga', 14], ['fantasy', 26], ['fantastique'...",/couv/CVT_Rose-eternelle_6800.jpg,106,4.35,...,0,0,0,0,0,1,0,0,0,0


In [233]:
7*8

56

In [291]:
X = new_b['title']
y = new_b.iloc[:,-56:]

In [236]:
y.columns

Index(['colere_0', 'colere_1', 'colere_2', 'colere_3', 'colere_4', 'colere_5',
       'colere_6', 'confiance_0', 'confiance_1', 'confiance_2', 'confiance_3',
       'confiance_4', 'confiance_5', 'confiance_6', 'honte_0', 'honte_1',
       'honte_2', 'honte_3', 'honte_4', 'honte_5', 'honte_6', 'joie_0',
       'joie_1', 'joie_2', 'joie_3', 'joie_4', 'joie_5', 'joie_6', 'peur_0',
       'peur_1', 'peur_2', 'peur_3', 'peur_4', 'peur_5', 'peur_6',
       'reflexion_0', 'reflexion_1', 'reflexion_2', 'reflexion_3',
       'reflexion_4', 'reflexion_5', 'reflexion_6', 'surprise_0', 'surprise_1',
       'surprise_2', 'surprise_3', 'surprise_4', 'surprise_5', 'surprise_6',
       'tristesse_0', 'tristesse_1', 'tristesse_2', 'tristesse_3',
       'tristesse_4', 'tristesse_5', 'tristesse_6'],
      dtype='object')

In [237]:
X.shape, y.shape

((908,), (908, 56))

## MultiLabelBinarizer

In [239]:
from sklearn.preprocessing import MultiLabelBinarizer

# Create MultiLabelBinarizer object
mlb = MultiLabelBinarizer() # se renseigner sur cette classe

# One-hot encode data
y_ = mlb.fit_transform(y)

In [241]:
y_.shape

(56, 24)

In [293]:
X_train, X_test , y_train, y_test = train_test_split(X, y, random_state=123)

In [294]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((681,), (227,), (681, 56), (227, 56))

### Test avec BERT uncased Multilingual

In [295]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3') # bert_multi_cased_preprocess
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4') # bert_multi_cased_L12_H768_A12

In [296]:
text_input = tf.keras.layers.Input(shape = (), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(56, activation = 'sigmoid', name = 'Dense')(dropout)

In [297]:
bert_uncased_multi= tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [298]:
bert_uncased_multi.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer_6 (KerasLayer)     {'input_mask': (Non  0           ['Inputs[0][0]']                 
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [299]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]

In [300]:
bert_uncased_multi.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [301]:
history = bert_uncased_multi.fit(X_train, y_train, epochs = 20)

Epoch 1/20
22/22 [==============================] - 16s 282ms/step - loss: 0.4018 - accuracy: 0.8284 - precision: 0.3962 - recall: 0.3868
Epoch 2/20
22/22 [==============================] - 6s 283ms/step - loss: 0.2862 - accuracy: 0.8683 - precision: 0.5578 - recall: 0.3719
Epoch 3/20
22/22 [==============================] - 6s 283ms/step - loss: 0.2792 - accuracy: 0.8702 - precision: 0.5675 - recall: 0.3792
Epoch 4/20
22/22 [==============================] - 6s 292ms/step - loss: 0.2763 - accuracy: 0.8706 - precision: 0.5695 - recall: 0.3805
Epoch 5/20
22/22 [==============================] - 6s 281ms/step - loss: 0.2745 - accuracy: 0.8715 - precision: 0.5751 - recall: 0.3785
Epoch 6/20
22/22 [==============================] - 6s 281ms/step - loss: 0.2738 - accuracy: 0.8719 - precision: 0.5825 - recall: 0.3603
Epoch 7/20
22/22 [==============================] - 6s 280ms/step - loss: 0.2726 - accuracy: 0.8712 - precision: 0.5678 - recall: 0.4059
Epoch 8/20
22/22 [======================

In [302]:
y_preds = np.round(bert_uncased_multi.predict(X_test))

8/8 [==============================] - 5s 310ms/step


In [303]:
y_preds.shape, y_test.shape

((227, 56), (227, 56))

In [304]:
y_preds[1]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.], dtype=float32)

In [305]:
y_test

,colere_0,colere_1,colere_2,colere_3,colere_4,colere_5,colere_6,confiance_0,confiance_1,confiance_2,...,surprise_4,surprise_5,surprise_6,tristesse_0,tristesse_1,tristesse_2,tristesse_3,tristesse_4,tristesse_5,tristesse_6
731,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
854,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
328,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
556,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
761,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
623,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
606,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
605,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [306]:
accuracy = accuracy_score(y_test, y_preds)

In [307]:
y_preds = np.round(bert_uncased_multi.predict(X_test))

print("Accuracy : ",accuracy_score(y_test, y_preds))

print("Precision Score : ",precision_score(y_test, y_preds, pos_label='positive', average='micro'))

print("Recall Score : ",recall_score(y_test, y_preds, pos_label='positive', average='micro'))

print("F1 Score : ",f1_score(y_test, y_preds, pos_label='positive', average='micro'))

8/8 [==============================] - 4s 298ms/step
Accuracy :  0.0
Precision Score :  0.5604217356042174
Recall Score :  0.3805066079295154
F1 Score :  0.4532633650377173


Il n'est pas possible de determiner correctement les sentiments en fonction du titre d'un livre car il n'y a pas assez de mots dans le titre qui peuvent être liés aux sentiments.

### Nouveau test avec les tags

In [309]:
books_tags = new_b[[col for col in new_b.columns if col.startswith('tag')][1:]]

X = books_tags
y = new_b.iloc[:,-56:]

In [323]:
X

,tag_roman,tag_classique,tag_autobiographie,tag_drogue,tag_littérature_américaine,tag_fantastique,tag_jeunesse,tag_humour,tag_littérature_italienne,tag_journalisme,...,tag_asexualité,tag_aromantisme,tag_licorne,tag_alopécie,tag_héritage,tag_mystère,tag_destin,tag_gangs,tag_littérature_algérienne,tag_identité
0,0,0,0,0,0,17,14,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,22,0,0,0,0,0,16,0,0,...,0,0,0,0,0,0,0,0,0,0
2,24,19,0,0,0,17,0,0,26,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,17,0,19,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21,0,0,0,0,26,23,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
903,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
904,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
905,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
906,0,0,0,0,0,18,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [326]:
y = y.to_numpy()

In [327]:
X_train, X_test , y_train, y_test = train_test_split(X, y, random_state=123)

In [328]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((681, 550), (227, 550), (681, 56), (227, 56))

In [315]:
bert_preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_preprocess/3') # bert_multi_cased_preprocess
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/4') # bert_multi_cased_L12_H768_A12

In [335]:
text_input = tf.keras.layers.Input(shape = (None, ), dtype = tf.string, name = 'Inputs')
preprocessed_text = bert_preprocessor(text_input)
embeed = bert_encoder(preprocessed_text)
dropout = tf.keras.layers.Dropout(0.1, name = 'Dropout')(embeed['pooled_output'])
outputs = tf.keras.layers.Dense(56, activation = 'sigmoid', name = 'Dense')(dropout)

ValueError: Exception encountered when calling layer "keras_layer_8" (type KerasLayer).

in user code:

    File "C:\Users\david\anaconda3\lib\site-packages\tensorflow_hub\keras_layer.py", line 237, in call  *
        result = smart_cond.smart_cond(training,

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional arguments (3 total):
        * <tf.Tensor 'inputs:0' shape=(None, None) dtype=string>
        * False
        * None
      Keyword arguments: {}
    
     Expected these arguments to match one of the following 4 option(s):
    
    Option 1:
      Positional arguments (3 total):
        * TensorSpec(shape=(None,), dtype=tf.string, name='inputs')
        * True
        * None
      Keyword arguments: {}
    
    Option 2:
      Positional arguments (3 total):
        * TensorSpec(shape=(None,), dtype=tf.string, name='inputs')
        * False
        * None
      Keyword arguments: {}
    
    Option 3:
      Positiona

In [317]:
bert_uncased_multi= tf.keras.Model(inputs = [text_input], outputs = [outputs])

In [318]:
bert_uncased_multi.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Inputs (InputLayer)            [(None,)]            0           []                               
                                                                                                  
 keras_layer_8 (KerasLayer)     {'input_word_ids':   0           ['Inputs[0][0]']                 
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [319]:
Metrics = [tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
           tf.keras.metrics.Precision(name = 'precision'),
           tf.keras.metrics.Recall(name = 'recall')
           ]

In [320]:
bert_uncased_multi.compile(optimizer ='adam',
               loss = 'binary_crossentropy',
               metrics = Metrics)

In [329]:
history = bert_uncased_multi.fit(X_train, y_train, epochs = 20)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\david\anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\david\anaconda3\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\david\anaconda3\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\david\anaconda3\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\david\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\david\AppData\Local\Temp\__autograph_generated_file434lu8fv.py", line 74, in tf__call
        ag__.if_stmt(ag__.not_(ag__.ld(self)._has_training_argument),